# Исследование NPS телекоммуникационной компании среди клиентов России

# Описание проекта

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite. 

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
9-10 баллов — «cторонники» (англ. promoters);
7-8 баллов — «нейтралы» (англ. passives);
0-6 баллов — «критики» (англ. detractors).
Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике. 
Интерпретируя результаты NPS-опросов, следует также помнить, что само значение мало о чём говорит. Однако исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса. 

**Используя дашборд, нужно ответить на вопросы:**

* Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?

* Какие группы пользователей наиболее лояльны к сервису? Какие менее?

* Какой общий NPS среди всех опрошенных?

* Как можно описать клиентов, которые относятся к группе cторонников?

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE 
           WHEN u.lt_day <= 365 THEN 'Новый'
           ELSE 'Старый'
           END AS is_new,
       u.age,
       CASE 
           WHEN u.gender_segment == 1 THEN 'Женщина'
           WHEN u.gender_segment == 0 THEN 'Мужчина'
           ELSE ' '
           END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       location.country,
       location.city,
       SUBSTR(age_segment.title, 4) AS age_segment,
       SUBSTR(traffic_segment.title, 4) AS traffic_segment,
       SUBSTR(lifetime_segment.title, 4) AS lifetime_segment,
       u.nps_score,
       CASE 
           WHEN u.nps_score >= 9 THEN 'Cторонник'
           WHEN u.nps_score >= 7 THEN 'Нейтрал'
           ELSE 'Критик'
           END AS nps_group
FROM user AS u
JOIN location ON u.location_id = location.location_id
JOIN age_segment ON u.age_gr_id = age_segment.age_gr_id
JOIN traffic_segment ON u.tr_gr_id = traffic_segment.tr_gr_id
JOIN lifetime_segment ON u.lt_gr_id = lifetime_segment.lt_gr_id;
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(5)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,Cторонник
1,A001WF,2344,Старый,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,Cторонник
2,A003Q7,467,Старый,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,Cторонник
3,A004TB,4190,Старый,44.0,Женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,Cторонник
4,A004XT,1163,Старый,24.0,Мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,Cторонник


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

**Дашборд на сайте:**
 https://public.tableau.com/app/profile/alexander.ryabyshenkov/viz/NPS_17177081827710/NPS_6?publish=yes

**Презентация:**
https://disk.yandex.ru/i/ncavtwBzWAz6ig